<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

## _*VQE algorithm: Application to quantum chemistry*_ 

The latest version of this notebook is available on https://github.com/QISKit/qiskit-tutorial.

***
### Contributors
Antonio Mezzacapo, Jay Gambetta

### Introduction

One of the most compelling possibilities of quantum computation is the the simulation of other quantum systems. Quantum simulation of quantum systems encompasses a wide range of tasks, including most significantly:
    
1. Simulation of the time evolution of quantum systems.

2. Computation of ground state properties. 

These applications are especially useful when considering systems of interacting fermions, such as molecules and strongly correlated materials. The computation of ground state properties of fermionic systems is the starting point for mapping out the phase diagram of condensed matter Hamiltonians. It also gives access to the key question of electronic structure problems in quantum chemistry - namely, reaction rates. The focus of this notebook is on molecular systems, which are considered to be the ideal bench test for early-stage quantum computers, due to their relevance in chemical applications despite relatively modest sizes. Formally, the ground state problem asks the following:

For some physical Hamiltonian *H*, find the smallest eigenvalue $E_G$, such that $H|\psi_G\rangle=E_G|\psi_G\rangle$, where $|\Psi_G\rangle$ is the eigenvector corresponding to $E_G$.
    
It is known that in general this problem is intractable, even on a quantum computer. This means that we cannot expect an efficient quantum algorithm that prepares the ground state of general local Hamiltonians. Despite this limitation, for specific Hamiltonians of interest it might be possible, given physical constraints on the interactions, to solve the above problem efficiently. Currently, at least four different methods exist to approach this problem:

1. Quantum phase estimation: Assuming that we can approximately prepare the state $|\psi_G\rangle$, this routine uses controlled implementations of the Hamiltonian to find its smallest eigenvalue. 
2. Adiabatic theorem of quantum mechanics: The quantum system is adiabatically dragged from being the ground state of a trivial Hamiltonian to the one of the target problem, via slow modulation of the Hamiltonian terms. 
3. Dissipative (non-unitary) quantum operation: The ground state of the target system is a fixed point. The non-trivial assumption here is the implementation of the dissipation map on quantum hardware. 
4. Variational quantum eigensolvers: Here we assume that the ground state can be represented by a parameterization containing a relatively small number of parameters.

In this notebook we focus on the last method, as this is most likely the simplest to be realized on near-term devices. 

The general idea is to define a parameterization $|\psi(\boldsymbol\theta)\rangle$ of quantum states, and minimize the energy 

$$E(\boldsymbol\theta) = \langle \psi(\boldsymbol\theta)| H |\psi(\boldsymbol\theta)\rangle,$$ 

The key ansatz is that the number of parameters $|\boldsymbol\theta^*|$ that minimizes the energy function scales polynomially with the size (e.g., number of qubits) of the target problem.  

Then, any local fermionic Hamiltonian can be mapped into a sum over Pauli operators $P_i$,  

$$H\rightarrow H_P = \sum_i^M w_i P_i,$$ 

and the energy corresponding to the state $|\psi(\boldsymbol\theta\rangle$, $E(\boldsymbol\theta)$, can be estimated by sampling the individual Pauli terms $P_i$ (or sets of them that can be measured at the same time) on a quantum computer: 

$$E(\boldsymbol\theta) = \sum_i^M w_i \langle \psi(\boldsymbol\theta)| P_i |\psi(\boldsymbol\theta)\rangle.$$ 

Last, some optimization technique must be devised in order to find the optimal value of parameters $\boldsymbol\theta^*$, such that $|\psi(\boldsymbol\theta^*)\rangle\equiv|\psi_G\rangle$.



### Fermionic Hamiltonians

The Hamiltonians describing systems of interacting fermions can be expressed in second quantization language, considering fermionic creation (annihilation) operators $a^\dagger_\alpha(a_\alpha)$, relative to the $\alpha$-th fermionic mode. In the case of molecules, the $\alpha$ labels stand for the different atomic or molecular orbitals. Within the second-quantization framework, a generic molecular Hamiltonian with $M$ orbitals can be written as 
$$H =H_1+H_2=\sum_{\alpha, \beta=0}^{M-1} t_{\alpha \beta} \, a^\dagger_{\alpha} a_{\beta} +\frac{1}{2}  \sum_{\alpha, \beta, \gamma, \delta = 0}^{M-1} u_{\alpha \beta \gamma \delta}\, a^\dagger_{\alpha} a^\dagger_{\gamma} a_{\delta} a_{\beta},$$
with the one-body terms representing the kinetic energy of the electrons and the potential energy that they experience in the presence of the nuclei, 
$$ t_{\alpha\beta}=\int d\boldsymbol x_1\Psi_\alpha(\boldsymbol{x}_1) \left(-\frac{\boldsymbol\nabla_1^2}{2}+\sum_{i} \frac{Z_i}{|\boldsymbol{r}_{1i}|}\right)\Psi_\beta (\boldsymbol{x}_1),$$
and their interactions via Coulomb forces 
$$ u_{\alpha\beta\gamma\delta}=\int\int d \boldsymbol{x}_1 d \boldsymbol{x}_2 \Psi_\alpha^*(\boldsymbol{x}_1)\Psi_\beta(\boldsymbol{x}_1)\frac{1}{|\boldsymbol{r}_{12}|}\Psi_\gamma^*(\boldsymbol{x}_2)\Psi_\delta(\boldsymbol{x}_2),$$
where we have defined the nuclei charges $Z_i$, the nuclei-electron and electron-electron separations $\boldsymbol{r}_{1i}$ and $\boldsymbol{r}_{12}$, the $\alpha$-th orbital wavefunction $\Psi_\alpha(\boldsymbol{x}_1)$, and we have assumed that the spin is conserved in the spin-orbital indices $\alpha,\beta$ and $\alpha,\beta,\gamma,\delta$.

### Molecules considered in this notebook and mapping to qubits

We consider in this notebook the optimization of two potential energy surfaces, for the hydrogen and lithium hydride molecules, obtained using the STO-3G basis. The molecular Hamiltonians are computed as a function of their interatomic distance, then mapped to two-(H$_2$) and four-(LiH$_2$) qubit problems, via elimination of core and high-energy orbitals and removal of $Z_2$ symmetries.

### Approximate universal quantum computing for quantum chemisty problems

In order to find the optimal parameters $\boldsymbol\theta^*$, we set up a closed optimization loop with a quantum computer, based on some stochastic optimization routine. Our choice for the variational ansatz is a deformation of the one used for the optimization of classical combinatorial problems, with the inclusion of $Z$ rotation together with the $Y$ ones. The optimization algorithm for fermionic Hamiltonians is similar to the one for combinatorial problems, and can be summarized as follows: 


1. Map the fermionic Hamiltonian $H$ to a qubit Hamiltonian $H_P$.
2. Choose the maximum depth of the quantum circuit (this could be done adaptively).
3. Choose a set of controls $\boldsymbol\theta$ and make a trial function $|\psi(\boldsymbol\theta)\rangle$. The difference with the combinatorial problems is the insertion of additional parametrized $Z$ single-qubit rotations.
4. Evaluate the energy $E(\boldsymbol\theta) = \langle\psi(\boldsymbol\theta)~|H_P|~\psi(\boldsymbol\theta)\rangle$ by sampling each Pauli term individually, or sets of Pauli terms that can be measured in the same tensor product basis.
5. Use a classical optimizer to choose a new set of controls.
6. Continue until the energy has converged, hopefully close to the real solution $\boldsymbol\theta^*$, and return the last value of $E(\boldsymbol\theta)$.
    

Note that, as opposed to the classical case, in the case of a quantum chemistry Hamiltonian one has to sample over non-computational states that are superpositions, and therefore take advantage of using a quantum computer in the sampling part of the algorithm. Motivated by the quantum nature of the answer, we also define a variational trial ansatz in this way: 

$$|\psi(\boldsymbol\theta)\rangle  = [U_\mathrm{single}(\boldsymbol\theta) U_\mathrm{entangler}]^m |+\rangle$$

where $U_\mathrm{entangler}$ is a collection of cPhase gates (fully entangling gates), $U_\mathrm{single}(\boldsymbol\theta) = \prod_{i=1}^n Y(\theta_{i})Z(\theta_{n+i})$ are single-qubit $Y$ and $Z$ rotation, $n$ is the number of qubits and $m$ is the depth of the quantum circuit. 



References and additional details:

[1] A. Kandala, A. Mezzacapo, K. Temme, M. Takita, M. Brink, J. M. Chow, and J. M. Gambetta, *Hardware-efficient Variational Quantum Eigensolver for Small Molecules and Quantum Magnets*, Nature 549, 242 (2017), and references therein.


In [ ]:
# useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import linalg as la
from functools import partial

# import QISKit
from qiskit import QuantumCircuit, register, execute, get_backend

# import Qconfig, set APIToken and API url
try:
    import sys
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
except Exception as e:
    print(e)
    qx_config = {
        "APItoken":"YOUR_TOKEN_HERE",
        "url":"https://quantumexperience.ng.bluemix.net/api"}

register(qx_config['APItoken'], qx_config['url'])

# import basic plot tools
from qiskit.tools.visualization import plot_histogram

# import optimization tools
from qiskit.tools.apps.optimization import trial_circuit_ryrz, SPSA_optimization, SPSA_calibration
from qiskit.tools.apps.optimization import hamiltonian_from_file, make_hamiltonian
from qiskit.tools.apps.optimization import eval_hamiltonian, group_paulis

# Ignore warnings due to chopping of small imaginary part of the energy 
import warnings
warnings.filterwarnings('ignore')

## Optimization of H$_2$ at bond length

In this first part of the notebook we show the optimization of the H$_2$ Hamiltonian in the STO-3G basis at the bond length of 0.735 Angstrom. After mapping it to a four-qubit system with a binary-tree transformation, two spin-parity symmetries are modded out, leading to a two-qubit Hamiltonian. The energy of the mapped Hamiltonian obtained is then minimized using the variational ansatz described in the introduction, and a stochastic perturbation simultaneous approximation (SPSA) gradient descent method. 

In [ ]:
# Set up optimisation
n = 2
m = 6
device = 'local_qasm_simulator'

initial_theta = np.random.randn(2*n*m)
backend = get_backend(device)
entangler_map = backend.configuration['coupling_map'] # the map of two-qubit gates with control at key and target at values
if entangler_map == 'all-to-all':
    entangler_map = {i: [j for j in range(n) if j != i] for i in range(n)}
shots = 1
max_trials = 100
ham_name = 'H2/H2Equilibrium.txt'
#ham_name = 'LIH/LiHEquilibrium.txt' #For optimization of LiH at bond length

In [ ]:
# Exact energy
pauli_list = hamiltonian_from_file(ham_name)
H = make_hamiltonian(pauli_list)
exact = np.amin(la.eig(H)[0]).real
print('The exact ground state energy is: ' + str(exact))
pauli_list_grouped=group_paulis(pauli_list)

In [ ]:
# Run optimization
def cost_function(H,n,m,entangler_map,shots,device,theta):
    return eval_hamiltonian(H,trial_circuit_ryrz(n,m,theta,entangler_map,None,False),shots,device).real

initial_c = 0.01
target_update = 2*np.pi*0.1
save_step = 20

if shots == 1:
    SPSA_params = SPSA_calibration(partial(cost_function,H,n,m,entangler_map,shots,device), \
                                   initial_theta,initial_c,target_update,25)
    output = SPSA_optimization(partial(cost_function,H,n,m,entangler_map,shots,device), \
                               initial_theta,SPSA_params,max_trials,save_step,1);
else:
    SPSA_params = SPSA_calibration(partial(cost_function,pauli_list_grouped,n,m,entangler_map,shots,device), \
                                   initial_theta,initial_c,target_update,25)
    output = SPSA_optimization(partial(cost_function,pauli_list_grouped,n,m,entangler_map,shots,device), \
                               initial_theta,SPSA_params,max_trials,save_step,1);


In [ ]:
plt.plot(np.arange(0, max_trials,save_step),output[2],label='E(theta_plus)')
plt.plot(np.arange(0, max_trials,save_step),output[3],label='E(theta_minus)')
plt.plot(np.arange(0, max_trials,save_step),np.ones(max_trials//save_step)*output[0],label='Final Energy')
plt.plot(np.arange(0, max_trials,save_step),np.ones(max_trials//save_step)*exact,label='Exact Energy')
plt.legend()
plt.xlabel('Trial state')
plt.ylabel('Energy')

## Optimizing the potential energy surface 

The optimization considered previously is now performed for two molecules, H$_2$ and LiH, for different interatomic distances, and the correspoding nuclei Coulomb repulsion is added in order to obtain a potential energy surface. 

In [ ]:
# Set molecule paramters

molecule = 'H2' # name of the molecule: options are H2 or LiH

if molecule == 'H2':
    n=2 # qubits
    Z1=1
    Z2=1
    min_distance=.2
    max_distance=4
    number_of_points=39
elif molecule == 'LiH':
    n=4 # qubits
    Z1=1
    Z2=3
    min_distance=.5
    max_distance=5
    number_of_points=46

In [ ]:
# Set optimisation parameters

run_optimization = True # Choose True or False. False just plots the exact potential energy surface

def cost_function(H,n,m,entangler_map,shots,device,theta):
    return eval_hamiltonian(H,trial_circuit_ryrz(n,m,theta,entangler_map,None,False),shots,device).real

m = 3 # depth (number of layers of sq gates - # entanglers is this number -1)
device = 'local_qasm_simulator'
initial_theta = np.random.randn(2*n*m)  # initial angles 
backend = get_backend(device)
entangler_map = backend.configuration['coupling_map'] # the map of two-qubit gates with control at key and target at values
if entangler_map == 'all-to-all':
    entangler_map = {i: [j for j in range(n) if j != i] for i in range(n)}
shots = 1
max_trials = 200

initial_c = 0.01
target_update = 2*np.pi*0.1
save_step = 250 # Setting larger than max_trials to suppress the output at every function call

In [ ]:
# Computing the potential energy surface

mol_distance = np.zeros(number_of_points)
coulomb_repulsion = np.zeros(number_of_points)
electr_energy = np.zeros(number_of_points)
electr_energy_optimized = np.zeros(number_of_points)

for i in range(number_of_points):

    # Coulomb Repulsion For Diatomic Molecules
    mol_distance[i] = np.around((min_distance+(max_distance-min_distance)*i/(number_of_points-1)),2)
    distance = mol_distance[i]/0.529177
    coulomb_repulsion[i] = Z1*Z2/distance
        
    # exact diagonalization
    ham_text = molecule+'/PESMap'+str(i)+'atdistance'+str(mol_distance[i])+'.txt'
    pauli_list = hamiltonian_from_file(ham_text)
    H = make_hamiltonian(pauli_list)
    pauli_list_grouped = group_paulis(pauli_list)
    eigen = la.eig(H)
    electr_energy[i] = np.amin(eigen[0])
           
    # optimization
    if run_optimization:
        print('\nOPTIMIZING HAMILTONIAN # '+str(i)+' AT INTERATOMIC DISTANCE ' + str(mol_distance[i]) + ' ANGSTROM\n')
            
        if shots !=1:
            H = group_paulis(pauli_list)
         
        SPSA_params = SPSA_calibration(partial(cost_function,H,n,m,entangler_map,shots,device),
                                       initial_theta,initial_c,target_update,25)
            
        electr_energy_optimized[i] = SPSA_optimization(partial(cost_function,H,n,m,entangler_map,shots,device),
                                                       initial_theta,SPSA_params,max_trials,save_step,1)[0];


In [ ]:
plt.plot(mol_distance,electr_energy+coulomb_repulsion,label='Exact')
if run_optimization:
    plt.plot(mol_distance,electr_energy_optimized+coulomb_repulsion,label='Optimized') 
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy')
plt.legend()